## XGBoost

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample
from xgboost import XGBClassifier
import time


def main():
    df = pd.read_csv("/content/drive/MyDrive/UNLV/train.csv")
    df.drop(["Id"], axis=1, inplace=True)

    df.columns = map(str.lower, df.columns)
    df.rename(columns={"married/single": "married_single"}, inplace=True)

    # --------------------
    # Category cols to num
    cate_cols = ["married_single", "profession", "house_ownership", "car_ownership", "city", "state"]

    for col in cate_cols:
        le = LabelEncoder()
        le = le.fit(df[col])
        df[col] = le.transform(df[col])

    print("Label Encoding-Done.")

    # --------------
    # Down sampling
    subset_0 = df[df["risk_flag"] == 0]
    subset_1 = df[df["risk_flag"] == 1]

    subset_0_downsampled = resample(subset_0,
                                    replace=False,
                                    n_samples=len(subset_1),
                                    random_state=42)

    df = pd.concat([subset_0_downsampled, subset_1])

    print("Down Sampling-Done.")

    X = df.drop(["risk_flag"], axis=1)
    y = df["risk_flag"].apply(lambda x: int(x))

    X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, stratify=y_train_val, random_state=42)

    xgb = XGBClassifier()

    # ---------------
    # Start: training
    start_time = time.time()

    xgb.fit(X_train, y_train)
    # -------------
    # End: training
    end_time = time.time()
    # Calculate training time
    elapsed_time = end_time - start_time

    y_val_pred = xgb.predict(X_val)
    y_test_pred = xgb.predict(X_test)

    val_auc = roc_auc_score(y_val, y_val_pred)
    test_auc = roc_auc_score(y_test, y_test_pred)

    print(f"Val AUC = {val_auc}")
    print(f"Test AUC = {test_auc}")
    print(f"Training Time: {elapsed_time // 60} min {elapsed_time % 60:.2f} sec")


if __name__ == "__main__":
    main()

Label Encoding-Done.
Down Sampling-Done.
Val AUC = 0.8284103031958838
Test AUC = 0.819904498124047
Training Time: 0.0 min 12.79 sec


## XGBoost with Pyspark

In [3]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=ad1fe413270c06b66c80bd36dbb8ed4f1e957be63560041f6d96d81533d300eb
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [7]:
!pip install sparkxgb

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.3/212.3 MB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 16.9 MB/s eta 0:00:00
  Created wheel for sparkxgb: filename=sparkxgb-0.1-py3-none-any.whl size=5630 sha256=1aa3e2c309c3c10e679dc7f35ed29c8842b62cb946022d8fd174863d882a384f
  Stored in directory: /root/.cache/pip/wheels/b7/0c/a1/786408e13056fabeb8a72134e101b1e142fc95905c7b0e2a71
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767590 sha256=393cf22d7c323b3898834a3514bcc2effbaaab32204e47ce5556573f9f59afe4
  Stored in directory: /root/.cache/pip/wheels/a0/3f/72/8efd988f9ae041f051c75e6834cd92dd6d13a726e206e8b6f3
Successfully built sparkxgb pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7
  Attempting uninstall: pyspark
    Found 

In [4]:
!pip install xgboost

In [11]:
from pyspark.sql import SparkSession
from xgboost.spark import SparkXGBClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import VectorAssembler, StandardScaler, StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql.functions import col as scol
from pyspark.sql.functions import rand

import time


def main():
    spark = SparkSession.builder.appName("UNLV").getOrCreate()

    df = spark.read.csv("/content/drive/MyDrive/UNLV/train.csv", header=True, inferSchema=True)
    df = df.drop("Id").withColumnRenamed("married/single", "married_single")
    df = df.toDF(*(col.lower() for col in df.columns))
    df = df.withColumn("risk_flag", scol("risk_flag").cast("integer"))

    # Category cols to num
    cate_cols = ["married_single", "profession", "house_ownership", "car_ownership", "city", "state"]

    indexers = [StringIndexer(inputCol=col, outputCol=col + "_idx").fit(df) for col in cate_cols]

    pipeline = Pipeline(stages=indexers)
    df = pipeline.fit(df).transform(df)
    df = df.drop(*cate_cols)

    print("Label Encoding-Done.")

    assembler = VectorAssembler(
        inputCols=[col for col in df.columns if col != "risk_flag"],
        outputCol="features"
    )
    df = assembler.transform(df)
    df = df.select(["features", "risk_flag"])

    # Down sampling
    pos_count = df.filter("risk_flag = 1").count()
    neg_df = df.filter("risk_flag = 0")
    sampled_neg_df = neg_df.orderBy(rand(seed=42)).limit(pos_count)
    df = sampled_neg_df.union(df.filter("risk_flag = 1"))
    df = df.withColumn("label", scol("risk_flag").cast("integer"))  # 이름 변경

    print("Down Sampling-Done.")

    # Define model
    train_val, test = df.randomSplit([0.8, 0.2], seed=42)
    train, val = train_val.randomSplit([0.8, 0.2], seed=42)

    xgb = SparkXGBClassifier(num_workers=10)

    # Start: training
    start_time = time.time()

    model = xgb.fit(train)

    # End: training
    end_time = time.time()
    # Calculate training time
    elapsed_time = end_time - start_time

    val_preds = model.transform(val)
    test_preds = model.transform(test)

    evaluator = BinaryClassificationEvaluator(labelCol="label", metricName="areaUnderROC")
    val_auc = evaluator.evaluate(val_preds)
    test_auc = evaluator.evaluate(test_preds)

    print(f"Val AUC = {val_auc:.3f}")
    print(f"Test AUC = {test_auc:.3f}")
    print(f"Elapsed Time: {elapsed_time // 60} min {elapsed_time % 60:.2f} sec")


if __name__ == "__main__":
    main()

Label Encoding-Done.


Down Sampling-Done.


KeyboardInterrupt: ignored